### Date: March 7, 2023

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import import_ipynb

import combined_date_cols
import data_cleaning
import data_merge
import extended_fu_prop
import extended_fu_variable
import met_stopping_rule_variable
import session_variable
import initial_follow_up
import total_follow_up
import rogue_session

from statistics import mean
from datetime import timedelta
from datetime import datetime

importing Jupyter notebook from combined_date_cols.ipynb
importing Jupyter notebook from data_cleaning.ipynb
importing Jupyter notebook from data_merge.ipynb
importing Jupyter notebook from extended_fu_prop.ipynb
importing Jupyter notebook from extended_fu_variable.ipynb
importing Jupyter notebook from met_stopping_rule_variable.ipynb
importing Jupyter notebook from session_variable.ipynb
importing Jupyter notebook from initial_follow_up.ipynb
importing Jupyter notebook from total_follow_up.ipynb
importing Jupyter notebook from rogue_session.ipynb


In [2]:
new_use = pd.read_csv("../new_use_not_clean.csv",
                     low_memory = False)
follow_up = pd.read_csv("../follow_up_not_clean.csv",
                       low_memory = False)

cancog_baseline = pd.read_excel("../camh_maac.xlsx")
maac_baseline = pd.read_excel("../maac.xlsx", "Enrolled MAAC Participants")

/var/folders/z3/7rdhq39j3bd3f8cqd48kghgc0000gn/T/ipykernel_5010/3593337333.py:7: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  maac_baseline = pd.read_excel("../maac.xlsx", "Enrolled MAAC Participants")


### Main data cleaning

In [3]:
new_use_data = data_cleaning.main_cleaning(data_frame = new_use,
                             time_point = "baseline", 
                             id_col = "First Name", 
                             email_col = "Email")
                        
follow_up_data = data_cleaning.main_cleaning(data_frame = follow_up, 
                              time_point = "baseline",
                              id_col = "First Name",
                              email_col = "Email")

### Drop New Use episodes without substance use

In [4]:
no_use = new_use_data[(new_use_data["Which of the following have you used within the past 2 hours?_Alcohol"] == "0") & (new_use_data["Which of the following have you used within the past 2 hours?_Cannabis"] == "0")]

In [5]:
new_use_data = new_use_data.drop(no_use.index, axis = 0).reset_index(drop = True)

In [6]:
nu_rename = {
    "&nbsp;": " ",
    " within the past 2 hours": "",
    "in the past 2 hours": "",
    " .": ".",
    " ?": "?",
    "\xa0": "",
    "(Click the link to view Marijuana Quantity image if necessary)": "",
    "  ": " ",
    " _": "_",
    " (check all that apply)": "",
    " other form": " form",
    "etc)": "etc.)",
    "drug": "Drug"
}

fu_rename = {
    "&nbsp;": " ",
    " since the last survey": "",
    "since the last survey": "",
    " _": "_",
    "other form": "form",
    "(Click the link below for Standard Marijuana Quantity image). ": "",
    " (check all that apply)": "",
    "Approximately h": "H",
    "Chocolate": "Chocolates",
    "positive effect": "positive effects",
    "consumed? ": "consumed?",
    "None of the Above": "None of the above", 
    "alcohol right now? ": "alcohol right now?",
    "What were your motivations for using cannabis?_To increase the positive effectss of alcohol": "What were your motivations for using cannabis?_To increase the positive effects of alcohol"
    
}

In [7]:
nu = new_use_data.columns.tolist()
fu = follow_up_data.columns.tolist()

for key in nu_rename: 
    nu = [x.replace(key, nu_rename[key]) for x in nu]
    
for key in fu_rename: 
    fu = [x.replace(key, fu_rename[key]) for x in fu]

In [8]:
nu_map = dict(zip(new_use_data.columns.tolist(), nu))
fu_map = dict(zip(follow_up_data.columns.tolist(), fu))

new_use_data = new_use_data.rename(mapper = nu_map,
                                   axis = 1)

follow_up_data = follow_up_data.rename(mapper = fu_map,
                                      axis = 1)

### Merging dataframes

In [9]:
merge = data_merge.merge_dataframes(new_use_data, follow_up_data)

In [10]:
merge_rename = {v: k for k, v in data_merge.merge_rename.items()}

In [11]:
merge.rename(columns = merge_rename, inplace = True)

In [12]:
merge = data_merge.update_id(data_frame = merge)

### Clean date format

In [13]:
merge_clean = combined_date_cols.clean_date_format(data_frame = merge, 
                                                   start_col = "survey_start",
                                                   submit_col = "survey_submit",
                                                   trigger_col = "trigger")

### Compute 'Session' variable

In [14]:
# Create column
baseline = merge_clean.copy() 
baseline["session"] = pd.Series(int)

# Store global variable
SESSION = 1

# Iterate through the dataframe
for i in baseline.index: 
    # Input data
    baseline.at[i, "session"] = SESSION
    # Check if at new participant data or at the end of the data
    if session_variable.check_participant_end(data = baseline, i = i): 
        SESSION = 1
    else:
        # Check if at the end of a use session
        if session_variable.check_session_end(data = baseline, i = i): 
            SESSION += 1

### Compute 'Day Number' variable

#### CanCog Data 

In [15]:
camh = cancog_baseline[["Study ID", "Date of Baseline(mm/dd/yyy)"]]

camh = camh.dropna(subset = ["Study ID", "Date of Baseline(mm/dd/yyy)"]).reset_index(drop = True)

camh["Study ID"] = camh["Study ID"].astype(int)

camh["date"] = camh["Date of Baseline(mm/dd/yyy)"]

#### MAAC Data

In [16]:
maac = maac_baseline[["Study ID", "Date of Baseline Interview (yyyy-mm-dd) "]] 

maac = maac.dropna().reset_index(drop = True)

maac["Study ID"] = maac["Study ID"].astype(int)

maac["Date of Baseline Interview (yyyy-mm-dd) "] = maac["Date of Baseline Interview (yyyy-mm-dd) "].astype(str)

for i in maac.index:
    maac.at[i, "date"] = maac.at[i, "Date of Baseline Interview (yyyy-mm-dd) "][:10]
    
maac["date"] = pd.to_datetime(maac["date"])

#### Combined dataframe

In [17]:
baseline_date = pd.concat([camh, maac])

baseline_date_data = baseline_date[["Study ID", "date"]].dropna().reset_index(drop = True)

baseline_date_data["baseline_date"] = baseline_date_data["date"] + timedelta(hours = 7)

#### Formatting with ID

In [18]:
baseline["baseline_date"] = pd.Series(int)

for i in baseline_date_data.index: 
    for j in baseline.index:
        if baseline_date_data.at[i, "Study ID"] == baseline.at[j, "id"]: 
            baseline.at[j, "baseline_date"] = baseline_date_data.at[i, "baseline_date"]

In [19]:
for i in baseline.index: 
    baseline.at[i, "day"] = (baseline.at[i, "survey_submit"] - baseline.at[i, "baseline_date"]).total_seconds()//(60*60*24)

#### Correcting start and end dates 

In [20]:
for i in baseline.index:
    if baseline.at[i, "day"] < 0: 
        baseline.at[i, "day"] = 0
    elif baseline.at[i, "day"] > 21: 
        baseline.at[i, "day"] = 21

### Compute the 'Time Since' variable

In [21]:
baseline = baseline.copy()
baseline["time_since_minutes"] = pd.Series(int)

# Iterate through data
for i in baseline.index: 
    # Store survey type 
    survey_type = baseline.at[i, "Survey Name"]
    if survey_type != "New Use Survey": 
        # Compute difference
        time_since = baseline.at[i, "survey_submit"] - trigger
        # Populate column
        baseline.at[i, "time_since_minutes"] = int((time_since.seconds)/60)
        # Update previous survey
        trigger = baseline.at[i, "survey_submit"]
    else:
        trigger = baseline.at[i, "survey_submit"]
        # Set time since to zero 
        baseline.at[i, "time_since_minutes"] = 0

### 'Initial Follow Up' variable

In [22]:
baseline = baseline.copy()

baseline['initial_follow_up'] = pd.Series(int) 
baseline[">_2_init"] = pd.Series(int)
count = 0

# Iterate through the data
for i in baseline.index: 
    # Row check
    if (i != (len(baseline.index)-1)):
        # Store survey type 
        survey_type = baseline.at[i, "Survey Name"]
        next_survey_type = baseline.at[i+1, "Survey Name"]
        # Type check
        if (survey_type == "New Use Survey"): 
            # Type check
            new_use_idx, submission_window = initial_follow_up.check_next_nu(next_survey_type, baseline, i) #store the index of the NU
            baseline.at[new_use_idx, ">_2_init"] = 0
            # store the submission window
            
        else: # if a FU survey 
            count = initial_follow_up.incr_count(baseline, i, submission_window, count) # stores the number of surveys within the 2.55 hr window
            count = initial_follow_up.populate_session(baseline, next_survey_type, count, new_use_idx)
    else: 
        initial_follow_up.populate_final(baseline, survey_type, submission_window,
                      new_use_idx, i, count)
        baseline.at[i, "initial_follow_up"] = 0

### 'Extended follow up' variable

In [23]:
# Create column
baseline["extended_fu"] = pd.Series(int)

# Initialize variables
SUBMISSION_WINDOW = baseline.at[i, "survey_submit"]
NU_IDX = 0
EXTENDED_FU = 0
INITIAL = 0
NO_INITIAL_COMPLETE = False
SUB_USE = False

# Iterate through data
for i in baseline.index:
    # Store survey type 
    survey_type = baseline.at[i, "Survey Name"]
    
    if survey_type == "New Use Survey":
        # Store New Use Session details
        NU_IDX, SUBMISSION_WINDOW = extended_fu_variable.store_nu_vals(data = baseline, i = i)
    
    if survey_type == "Follow Up Surveys":
        # Check if the Follow Up is within time and whether substances are used
        IN_TIME = extended_fu_variable.check_within_time(data = baseline, i = i, submission_window = SUBMISSION_WINDOW)
        NO_SUB = extended_fu_variable.check_sub_use_efu(data = baseline, i = i)
        
        if IN_TIME:
            if not NO_SUB:
                SUB_USE = True
            
            # Count the number of initial surveys (2) are completed regardless of substance use
            INITIAL +=1
        else: 
            NO_INITIAL_COMPLETE = True
    
    # Substance(s) used in initial surveys
    if INITIAL != 0 and SUB_USE:
        EXTENDED_FU = 1
    
    # No substance(s) used in initial surveys
    if INITIAL != 0 and not SUB_USE:
        EXTENDED_FU = 0
        
    # No initial surveys completed
    if NO_INITIAL_COMPLETE and INITIAL == 0: 
        EXTENDED_FU = np.nan
    
    # Input details to the New Use Survey index
    if extended_fu_variable.check_end(data = baseline, i = i): 
        baseline.at[NU_IDX, "extended_fu"] = EXTENDED_FU
        # Reset
        EXTENDED_FU = 0
        INITIAL = 0
        NO_INITIAL_COMPLETE = False
        SUB_USE = False

### 'Met Stopping Rule' variable

In [24]:
baseline["met_stop_rule"] = np.nan
baseline["one_met_stop_rule"] = pd.Series(int)

nu_idx = 0 
count = 0
time_window = baseline.at[0, "survey_submit"]


for i in baseline.index: 
    survey_type = baseline.at[i, "Survey Name"]
    # If a NU session, store the variables
    nu_idx, time_window, count  = met_stopping_rule_variable.store_nu_vars(data = baseline,
                                                                           survey_type = survey_type,
                                                                           i = i,
                                                                           nu_idx = nu_idx, 
                                                                           count = count, 
                                                                           time_window = time_window)
    # Check if within time window
    if met_stopping_rule_variable.check_fu_window(data = baseline,
                       i = i, 
                       time_window = time_window,
                       survey_type = survey_type):
        
        # Check if substances were used
        count = met_stopping_rule_variable.check_sub_use_msr(data = baseline, 
                                                             i = i, 
                                                             count = count)
        
        # Update time window 
        time_window = met_stopping_rule_variable.update_time_window(data = baseline, 
                                                                    i = i)
    # Check if at the end of the session
    if met_stopping_rule_variable.check_session_data_end(data = baseline, 
                                                         i = i):
        met_stopping_rule_variable.populate_nu_idx(data = baseline, 
                                                   nu_idx = nu_idx, 
                                                   count = count)

### 'Total Follow Up' variables

In [25]:
# Create the column
baseline["total_fu_num"] = pd.Series(int)

# Create variables
nu_idx = 0 
time_window = baseline.at[0, "survey_submit"]
total = 0

for i in baseline.index: 
    # Store the survey type
    survey_type = baseline.at[i, "Survey Name"]
    # If a NU session, store variables 
    nu_idx, time_window, total = total_follow_up.check_nu_survey(data = baseline, 
                                                 i = i,
                                                 nu_idx = nu_idx, 
                                                 time_window = time_window, 
                                                 total = total)
    # If a FU survey, check within time
    if met_stopping_rule_variable.check_fu_window(data = baseline, 
                       survey_type = survey_type,
                       i = i, 
                       time_window = time_window): 
        # Update total 
        total += 1
        # Updates the time window 
        time_window = met_stopping_rule_variable.update_time_window(data = baseline, 
                                         i = i)
    # Checks if at the end of the session
    if met_stopping_rule_variable.check_session_data_end(data = baseline,
                         i = i):
        # Populate the 'total_fu_num' variable
        total_follow_up.populate_total_fu(data = baseline, 
                          nu_idx = nu_idx,
                          total = total)

### 'Total Follow Ups Needed' variable

In [26]:
# Create column
baseline["total_fu_needed"] = pd.Series(int)

for i in baseline.index: 
    # Check if stop rule was met
    if baseline.at[i, "met_stop_rule"] == 1:
        baseline.at[i, "total_fu_needed"] = baseline.at[i, "total_fu_num"]
    else:
        # Check if one survey met the stop rule
        if baseline.at[i, "one_met_stop_rule"] == 1:
            baseline.at[i, "total_fu_needed"] = baseline.at[i, "total_fu_num"]+1 
        else:
            baseline.at[i, "total_fu_needed"] = baseline.at[i, "total_fu_num"]+2

### 'Total Follow Up Proportion' variable

In [27]:
# Create column
baseline["total_fu_prop"] = pd.Series(int)

for i in baseline.index:
    # Compute the proportion of the session
    baseline.at[i, "total_fu_prop"] = baseline.at[i, "total_fu_num"]/baseline.at[i, "total_fu_needed"]

### 'Number of Extended Follow Ups Completed' variable

In [28]:
# Create column
baseline["num_ex_fu_completed"] = np.nan

for i in baseline.index:
    # If session included an extended Follow Up
    if baseline.at[i, "extended_fu"] == 1:
        total = baseline.at[i, "total_fu_num"] 
        initial = baseline.at[i, "initial_follow_up"]
        baseline.at[i, "num_ex_fu_completed"] = total - initial
        # Handle error cases
        if baseline.at[i, "num_ex_fu_completed"] < 0: 
            baseline.at[i, "num_ex_fu_completed"] = 0

### 'Extended Follow Up Proportion' variable

In [29]:
baseline["ex_fu_prop"] = pd.Series(int)

for i in baseline.index:
    if extended_fu_prop.check_survey(data = baseline, i = i): 
        if extended_fu_prop.check_extended_fu(data = baseline, i = i):
            total = int(baseline.at[i, "num_ex_fu_completed"])
            extended_fu_prop.check_stop_rule(data = baseline, i = i)
            extended_fu_prop.check_half_stop(data = baseline, i = i, total = total)
        else: 
            baseline.at[i, "ex_fu_prop"] = None

### New Use dataset

In [30]:
baseline_nu = baseline[baseline["Survey Name"] == "New Use Survey"].reset_index(drop = True)

### Average number of initial follow ups per New Use

In [31]:
baseline_nu["init_fu_prop"] = baseline_nu["initial_follow_up"]/2

init_fu_prop = baseline_nu[["id", "init_fu_prop"]].groupby("id").mean()

init_fu = init_fu_prop.reset_index()

baseline["init_fu_prop"] = pd.Series(int)

for i in init_fu.index: 
    for j in baseline.index: 
        
        if init_fu.at[i, "id"] == baseline.at[j, "id"]: 
            baseline.at[j, "init_fu_prop"] = init_fu.at[i, "init_fu_prop"]

### Average number of total Follow Up surveys per New Use

In [32]:
avg_total_fu = baseline_nu[["id", "total_fu_num", "total_fu_needed"]].groupby("id").sum().reset_index()

avg_total_fu["avg_total_fu"] = avg_total_fu["total_fu_num"]/avg_total_fu["total_fu_needed"]

baseline["total_fu_prop"] = pd.Series(int)

for i in avg_total_fu.index:
    for j in baseline.index:
        
        if avg_total_fu.at[i, "id"] == baseline.at[j, "id"]: 
            baseline.at[j, "total_fu_prop"] = avg_total_fu.at[i, "avg_total_fu"]

### Proportion of total Follow Up surveys per New Use

In [33]:
total_prop = baseline_nu[["id", "total_fu_prop"]].groupby("id").mean()
total_prop = total_prop.reset_index()

baseline["avg_fu_prop_part"] = pd.Series(int)

for i in total_prop.index: 
    for j in baseline.index: 
        
        if total_prop.at[i, "id"] == baseline.at[j, "id"]: 
            baseline.at[j, "avg_fu_prop_part"] = total_prop.at[i, "total_fu_prop"]

### Average number of extended Follow Up surveys per New Use

In [34]:
ex_fu = baseline_nu[["id", "num_ex_fu_completed"]].groupby("id").mean()
ex_fu = ex_fu.reset_index()

baseline["ex_fu_avg"] = pd.Series(int)

for i in ex_fu.index: 
    for j in baseline.index: 
        
        if ex_fu.at[i, "id"] == baseline.at[j, "id"]: 
            baseline.at[j, "ex_fu_avg"] = ex_fu.at[i, "num_ex_fu_completed"]

### Average extended Follow Up proportion

In [35]:
prop_ex_fu = baseline_nu[["id", "ex_fu_prop"]].groupby("id").mean()
prop_ex_fu = prop_ex_fu.reset_index()

baseline["ex_fu_prop_part"] = pd.Series(int)

for i in prop_ex_fu.index:
    for j in baseline.index: 
        
        if prop_ex_fu.at[i, "id"] == baseline.at[j, "id"]:
            baseline.at[j, "ex_fu_prop_part"] = prop_ex_fu.at[i, "ex_fu_prop"]

### 'Rogue Session' variable

In [36]:
baseline["session_rogue"] = pd.Series(int)
rogue_sess = False

for i in baseline.index: 
    rogue_sess = rogue_session.populate_session_rogue(data = baseline,
                                                      idx = i, 
                                                      rogue_sess = rogue_sess)

### Output

In [37]:
baseline.to_excel('merge_v3.xlsx')